# Mount my Google Drive where I stored the data

In [20]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Grab YOLOv3 model from git repo



In [21]:
!git clone https://github.com/ultralytics/yolov3.git

fatal: destination path 'yolov3' already exists and is not an empty directory.


# Installing requirements

In [ ]:
!cd yolov3 && pip install -r requirements.txt

# Download config files from my github repo

In [23]:
!rm -rf ./bird_detect
!git clone https://github.com/crabius/bird_detect

Cloning into 'bird_detect'...
remote: Enumerating objects: 1419, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1419 (delta 43), reused 75 (delta 36), pack-reused 1329
Receiving objects: 100% (1419/1419), 76.39 MiB | 46.72 MiB/s, done.
Resolving deltas: 100% (109/109), done.


# Move training, testing and validation images and labels to correct place

In [27]:
!cp ./drive/MyDrive/bird_data/trains.zip ./trains.zip

In [ ]:
!unzip ./trains.zip

In [29]:
!rm -rf ./yolov3/train
!mkdir ./yolov3/train

!rm -rf ./yolov3/test
!mkdir ./yolov3/test

!rm -rf ./yolov3/val
!mkdir ./yolov3/val

!mkdir ./yolov3/train/images
!mkdir ./yolov3/train/labels

!mkdir ./yolov3/test/images
!mkdir ./yolov3/test/labels

!mkdir ./yolov3/val/images
!mkdir ./yolov3/val/labels

!cp -r ./trains/train/* ./yolov3/train/images
!cp -r ./trains/train_out/* ./yolov3/train/labels

!cp -r ./drive/MyDrive/bird_data/val/* ./yolov3/val/images
!cp -r ./drive/MyDrive/bird_data/val_out/* ./yolov3/val/labels

!cp -r ./drive/MyDrive/bird_data/test/vanilla/* ./yolov3/test/images
!cp -r ./drive/MyDrive/bird_data/test_out/* ./yolov3/test/labels

# Double check for test train val leakage

In [30]:
import os
import collections

def dupes():
    """
    Just double checking if train test val have any overlapping files
    """
    folds = ["./yolov3/test/images","./yolov3/val/images","./yolov3/train/images"]
    files=[]
    for folder in folds:
        files.append(os.listdir(folder))
    res = []
    for folder in files:
        for file in folder:
            res.append(file)
    #check if any files appear twice
    print("duplicates: ",
        [item for item,count in collections.Counter(res).items() if count > 1]
    )

dupes()

duplicates:  []


# Grab custom.yaml file from git repo to define my dataset format, classes etc

In [31]:
!mv ./bird_detect/custom.yaml ./yolov3/data/custom.yaml

 # Use tensorboard for monitoring training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./yolov3/runs/train

# Finetune our pretrained model on our train images for 75 epochs

In [ ]:
!cd ./yolov3 && python3 train.py --weights yolov3-spp.pt --data "data/custom.yaml" --workers 8 --device 0 --batch-size 4 --img 416 --epochs 75

# Optional: move folder of previous run over from drive

In [32]:
!mkdir ./yolov3/runs
!mkdir ./yolov3/runs/train
!cp -r './drive/MyDrive/v3_75_epochs' ./yolov3/runs/train/

mkdir: cannot create directory ‘./yolov3/runs’: File exists
mkdir: cannot create directory ‘./yolov3/runs/train’: File exists


# Run object detection on image test set
need to specify weights with --weights based on where they are saved

In [ ]:
!cd yolov3 && python detect.py --weights 'runs/train/v3_75_epochs/weights/best.pt' --imgsz 400 --line-thickness 1 --conf-thres 0.2 --iou-thres 0.35 --source "./test/images/"

# Copy the results so I can look at them later

In [16]:
!cp -r ./yolov3/runs/detect/exp13 ./drive/MyDrive/v3_test

cp: cannot stat './yolov3/runs/detect/exp13': No such file or directory


# Download videos of birds to run inference on

In [35]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytube-12.1.0-py3-none-any.whl (56 kB)


In [36]:
from pytube import YouTube
#tui
YouTube('https://m.youtube.com/watch?v=ucvO-gphZng').streams[1].download()
#takahe
YouTube('https://m.youtube.com/watch?v=XRC4VATT9i8').streams[1].download()
#kea 
YouTube('https://m.youtube.com/watch?v=OXqhYAwSask').streams[1].download()

'/content/Kea Unscrewed.mp4'

# rename them

In [37]:
!mv *Kea* kea.mp4
!mv *Takahe* takahe.mp4
!mv *Tui* tui.mp4

# Now run inference on bird videos (have to tweak confidence threshold and iou threshold manually to get best results in different contexts)

In [ ]:
#!cd yolov3 && python detect.py --weights 'runs/train/v3_75_epochs/weights/best.pt' --source "../kea.mp4" --conf-thres 0.53
!cd yolov3 && python detect.py --weights './runs/train/v3_75_epochs/weights/best.pt' --source "../tui.mp4" --conf-thres 0.50
#!cd yolov3 && python detect.py --weights 'runs/train/v3_75_epochs/weights/best.pt' --source "../takahe.mp4" --conf-thres 0.53

# Copy the results back to my drive so I can watch them later
Have to manually adjust the folder directories, will be the last 3 folders in ./yolov3/runs/detect/

In [39]:
#!cp ./yolov3/runs/detect/exp14/kea.mp4 ./drive/MyDrive/kea_v3.mp4
!cp ./yolov3/runs/detect/exp4/tui.mp4 ./drive/MyDrive/tui_v3.mp4
#!cp ./yolov3/runs/detect/exp16/takahe.mp4 ./drive/MyDrive/takahe_v3.mp4